In [ ]:
!pip install --user keras
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!git clone https://github.com/ziadloo/attention_keras.git
!pip install keras-attention
!pip install keras-self-attention
!pip install keras==2.2.4        
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install tensorflow==1.14.0

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-zjdzzl5k
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-zjdzzl5k
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=680a06ad56df5e1e8795fd85c366e3ce2dcb43d56a0e2627f0422b514772d8c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-zt_i7whi/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Cloning into 'attention_keras'...
remote: Enumerating objects: 199, done.
remote: Total 199 (delta 0), reused 0 (delta 0), pack-reused 199
Receiving objects: 100% (199/199), 47.20 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (90/90), done.
  Created wheel for keras-self-attention: filename=keras_self_attention-0.49.0-cp37-none-any.whl size=19468 sha256=0f854f75f86885897b3ec817482a99935f31125bed25c68cf9c53eb3edd0fa29
  Stored in directory: /root/.cache/pip/wheels/6

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import numpy
from collections import Counter
from keras.models import *
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dropout, Dense,concatenate
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras_contrib.datasets import conll2000
from keras_self_attention import SeqSelfAttention
from collections import Counter
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

Mounted at /content/drive


In [ ]:

EMBED_DIM = 400
BiRNN_UNITS = 400
max_len_char=-1
max_len=None
chunking_file_path = '/content/drive/My Drive/explo/bhoj_chunk.txt'
output_file = 'output.txt'
def load_data(chunking_file_path, min_freq=1):

    file_train = _parse_data(open(chunking_file_path))
    print (len(file_train))
    count1=0
    for i in file_train:
        count1=count1+len(i)
    print (count1)
    word_counts = Counter(row[0].lower() for sample in file_train for row in sample if len(row)>=3)
    vocab = ['<pad>', '<unk>']
    vocab += [w for w, f in iter(word_counts.items()) if f >= min_freq]
    pos_tags = sorted(list(set(row[1] for sample in file_train for row in sample if len(row)>=3)))
    chunk_tags = sorted(list(set(row[2] for sample in file_train for row in sample if len(row)>=3)))
    characters = sorted(list(set(i for sample in file_train for row in sample if len(row)>=3 for i in row[0])))
    characters.insert(0,'<unk>')
    characters.insert(0,'<pad>')
    pos_tags.insert(0,'<unk>')
    pos_tags.insert(0,'<pad>')
    global max_len_char
    for sample in file_train:
        for row in sample:
            if len(row) >= 3:
                max_len_char=max(max_len_char,len(row[0]))
    train = _process_data(file_train, vocab, pos_tags, chunk_tags,characters)
    return train, (vocab, pos_tags, chunk_tags,characters)
def _parse_data(fh):
    string = fh.read()
    # print(string)
    data = []
    for sample in string.strip().split('\n\n'):
        data.append([row.split() for row in sample.split('\n')])
    fh.close()
    return data
def pad_words(l,max_len_char):
    length=len(l)
    l1=[0 for i in range(max_len_char-length)]
    l=l1+l
    return l
def _process_data(data, vocab, pos_tags, chunk_tags, characters, onehot=False):
    global max_len
    if max_len is None:
        max_len = max(len(s) for s in data)
    word2idx = dict((w, i) for i, w in enumerate(vocab))
    x = [[word2idx.get(w[0].lower(), 1) for w in s if len(w)>=3] for s in data]
    y_pos = [[pos_tags.index(w[1]) for w in s if len(w)>=3] for s in data]
    y_chunk = [[chunk_tags.index(w[2]) for w in s if len(w)>=3] for s in data]
    defaultvalue=[0 for i in range(max_len_char)]
    x = pad_sequences(x, max_len)
    y_pos = pad_sequences(y_pos, max_len, value=0)
    y_chunk = pad_sequences(y_chunk, max_len, value=0)

    if onehot:
        y_pos = numpy.eye(len(pos_tags), dtype='float32')[y]
        y_chunk = numpy.eye(len(chunk_tags), dtype='float32')[y]
    else:
        y_pos = numpy.expand_dims(y_pos, 2)
        y_chunk = numpy.expand_dims(y_chunk, 2)
    return x, y_pos, y_chunk
def tocharacter(characters,vocab,X_train):
    ''' Function to create word embedding into character embedding'''
    char2idx= dict((w,i) for i,w in enumerate(characters))
    idx2word= dict((i, w) for i, w in enumerate(vocab)) 
    l=[]
    for s in X_train:
        l1=[]
        for w in s:
            if (idx2word[w]=='<pad>'):
                l1.append([0]*max_len_char)
                continue
            if (idx2word[w]=='<unk>'):
                l1.append([1]*max_len_char)
                continue
            l2=[]
            for c in idx2word[w]:
                l2.append(char2idx.get(c,1))
            l2=pad_words(l2,max_len_char)
            l1.append(l2)
        l.append(l1)
    return numpy.asarray(l)
def classification_report(y_true, y_pred, labels):
    '''Similar to the one in sklearn.metrics,
    reports per classs recall, precision and F1 score'''
    y_true = numpy.asarray(y_true).ravel()
    y_pred = numpy.asarray(y_pred).ravel()
    corrects = Counter(yt for yt, yp in zip(y_true, y_pred) if yt == yp)
    y_true_counts = Counter(y_true)
    y_pred_counts = Counter(y_pred)
    report = ((lab,  # label
               corrects[i] / max(1, y_true_counts[i]),  # recall
               corrects[i] / max(1, y_pred_counts[i]),  # precision
               y_true_counts[i]  # support
               ) for i, lab in enumerate(labels))
    report = [(l, r, p, 2 * r * p / max(1e-9, r + p), s) for l, r, p, s in report]

    print('{:<15}{:>10}{:>10}{:>10}{:>10}\n'.format('',
                                                    'recall',
                                                    'precision',
                                                    'f1-score',
                                                    'support'))
    formatter = '{:<15}{:>10.2f}{:>10.2f}{:>10.2f}{:>10d}'.format
    for r in report:
        print(formatter(*r))
    print('')
    report2 = list(zip(*[(r * s, p * s, f1 * s) for l, r, p, f1, s in report]))
    N = len(y_true)
    print(formatter('avg / total',
                    sum(report2[0]) / N,
                    sum(report2[1]) / N,
                    sum(report2[2]) / N, N) + '\n')
    actual = Counter(y_true)
    del actual[-1]
    accuracy = sum(corrects.values()) / sum(actual.values())
    print('Accuracy:', accuracy)

In [ ]:
train, voc = load_data(chunking_file_path)
(X,x_pos,y) = train
(vocab, pos_tags, class_labels,characters) = voc
f2=open("vocabs.txt","w")
f2.write(" ".join(vocab)+'\n')
f2.write(" ".join(characters)+'\n')
f2.write(" ".join(pos_tags)+'\n')
f2.write(" ".join(class_labels)+'\n')
f2.flush()
f2.close()
X_char=tocharacter(characters,vocab,X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2018)
X_char_train, X_char_test, __ , __ = train_test_split(X_char, y, test_size=0.3,random_state=2018)
x_pos_train,x_pos_test,__, ___ = train_test_split(x_pos, y, test_size=0.3,random_state=2018)
print('==== training BiLSTM-CRF ====')
print(max_len_char)
print(max_len)

6327
87729
==== training BiLSTM-CRF ====
23
369


In [ ]:
from keras import backend as K
# word embedding
word_in=Input(shape=(X_train.shape[1],),name='word_in')
emb_word=(Embedding(len(vocab), EMBED_DIM//4, mask_zero=True))(word_in)

print(X_train.shape[1])

369


In [ ]:
# charcter embedding
char_in = Input(shape=(X_char_train.shape[1],X_char_train.shape[2]), name='char_in')
emb_char=(TimeDistributed(Embedding(input_dim=len(characters),output_dim=10,input_length=max_len_char,mask_zero=True)))(char_in)
char_enc=(TimeDistributed(LSTM(units=EMBED_DIM//4,return_sequences=False)))(emb_char)

# pos_tag embedding 
pos_in=Input(shape=(x_pos_train.shape[1],),name='pos_in')
pos_word=(Embedding(len(pos_tags), EMBED_DIM//2, mask_zero=True))(pos_in)

# concatenating them word+char+pos
x = concatenate([emb_word, char_enc, pos_word])

#passing to Bi-LSTM layer
o=(Bidirectional(LSTM(BiRNN_UNITS // 2, return_sequences=True, dropout=0.2)))(x)
#print(o)

#Self attention layer
o = (SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       kernel_regularizer=keras.regularizers.l2(1e-4),
                       bias_regularizer=keras.regularizers.l1(1e-4),
                       attention_regularizer_weight=1e-4,
                       name='Attention'))(o)



In [ ]:
#CRF layer
crf = CRF(len(class_labels), sparse_target=True,name='crf')
o=(crf)(o)
model=Model([word_in,char_in,pos_in],o)
model.compile('adam', loss=crf_loss, metrics=[crf_viterbi_accuracy])

In [ ]:
EPOCHS = 5
model.fit([X_train,np.array(X_char_train).reshape(len(X_char_train),max_len,max_len_char),x_pos_train.reshape(len(x_pos_train),max_len)],np.array(y_train).reshape(len(y_train), max_len, 1),batch_size=32, epochs=EPOCHS, validation_split=0.1, verbose=1)
model.save('/content/drive/My Drive/explo/bhoj_explo_chunk.bin')

Train on 3985 samples, validate on 443 samples
Epoch 1/5
3985/3985 [==============================] - 1382s 347ms/step - loss: 172.7714 - crf_viterbi_accuracy: 0.3522 - val_loss: 145.2792 - val_crf_viterbi_accuracy: 0.4196
Epoch 2/5
3985/3985 [==============================] - 1355s 340ms/step - loss: 171.8562 - crf_viterbi_accuracy: 0.6717 - val_loss: 144.2400 - val_crf_viterbi_accuracy: 0.8264
Epoch 3/5
3985/3985 [==============================] - 1358s 341ms/step - loss: 171.2712 - crf_viterbi_accuracy: 0.8968 - val_loss: 143.9212 - val_crf_viterbi_accuracy: 0.9233
Epoch 4/5
3985/3985 [==============================] - 1375s 345ms/step - loss: 171.1501 - crf_viterbi_accuracy: 0.9216 - val_loss: 143.8838 - val_crf_viterbi_accuracy: 0.9223
Epoch 5/5
3985/3985 [==============================] - 1381s 346ms/step - loss: 171.0862 - crf_viterbi_accuracy: 0.9309 - val_loss: 143.8598 - val_crf_viterbi_accuracy: 0.9217


In [ ]:
print(len(X_char_train))
arr = np.array(X_char_train)
print(arr.shape)

print(len(X_char_test))
arr_test = np.array(X_char_test)
print(arr_test.shape)

4428
(4428, 369, 23)
1899
(1899, 369, 23)


In [ ]:
#validation
model_json = model.to_json()
with open("model_mai_only.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/explo/model_only_bhoj.h5")
print("Saved model to disk")
l4=[0]*max_len_char
l4=numpy.asarray(l4)
y= model.predict([X_test,np.array(X_char_test).reshape((len(X_char_test),max_len, max_len_char)),x_pos_test.reshape(len(x_pos_test),max_len)])
y= y.argmax(-1)
y_pred=[]
test_y_true=[]
ctest_y_pred=[]
ctest_y_true=[]

Saved model to disk


In [ ]:
#removing padding from the validation
for i in range(len(X_test)):
    l=[]
    l1=[]
    for j in range(len(X_test[i])):
        if (X_test[i][j]==0):
            continue
        l.append(y[i][j])
        l1.append(y_test[i][j][0])
        ctest_y_pred.append(y[i][j])
        ctest_y_true.append(y_test[i][j][0])
    y_pred.append(l)
    test_y_true.append(l1)

#converting to numpy array
test_y_pred=numpy.asarray(y_pred)
test_y_true=numpy.asarray(test_y_true)

#writing output for validation
f1=open(output_file,'w')
for i in range(len(test_y_pred)):
    for j in range(len(test_y_pred[i])):
        s='|'+'\t'+'NN'+'\t'+str(class_labels[test_y_pred[i][j]])+'\t'+str(class_labels[test_y_true[i][j]])
        f1.write(s+'\n')
    f1.write('\n')
f1.flush()
f1.close()
print('\n---- Result of BiLSTM-CRF ----\n')
classification_report(ctest_y_true, ctest_y_pred, class_labels)


---- Result of BiLSTM-CRF ----

                   recall precision  f1-score   support

B-BLK                0.74      0.68      0.71       355
B-CCP                0.99      0.95      0.97       747
B-JJP                0.00      0.00      0.00       160
B-NEGP               0.01      1.00      0.03        73
B-NNP                0.00      0.00      0.00         0
B-NP                 0.97      0.92      0.94      6846
B-NST                0.00      0.00      0.00         0
B-PRP                0.00      0.00      0.00         0
B-RBP                0.09      0.58      0.15        81
B-RP                 0.00      0.00      0.00         0
B-VGF                0.96      0.89      0.92      3205
B-VGINF              0.00      0.00      0.00         9
B-VGNF               0.29      0.53      0.38       394
B-VGNN               0.00      0.00      0.00         4
I-BLK                0.27      0.57      0.37        63
I-CCP                0.00      0.00      0.00        11
I-JJP         

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## **vec mapping**

In [ ]:
# below used file is the file same as used for training of data ( word - NER tag ) type
f = open ('/content/drive/My Drive/explo/bhoj_chunk.txt','r')
lines = f.readlines()
print(lines[0].split('\t'))

words = []
sents = []
sent = []
for line in lines:
  line = line.split('\t')
  if line[0]!='\n':
    word = line[0].split('\ufeff')
    if len(word)>1:
      word = word[1]
    else:
      word = word[0]
    if word == '।':
      sent.append(word)
      sents.append(sent)
      sent=[]
    else:
      sent.append(word)

from gensim.models import Word2Vec
word2vec = Word2Vec(sents, size=100,window=5,min_count=2, negative=10,sg=1)
f.close()

['\ufeffसुकन्या', 'NNP', 'B-NP\n']


In [ ]:
print(len(word2vec.wv.vocab))

5836


In [ ]:
f = open ('/content/drive/My Drive/explo/bhoj.txt','r')
embeddings_index={}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
embedding_matrix = np.zeros((len(vocab), 100))
i = 0
for word in vocab:
    # values = line.split()
    # word = values[0]
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    i=i+1

In [ ]:
# model = Sequential()
# pretrained_weights = word2vec.wv.syn0
# model.add(Embedding(input_dim=len(word2vec.wv.vocab)+1, output_dim=100,weights =[embedding_matrix] ,mask_zero=True))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [ ]:
from keras import backend as K
# word embedding

word_in=Input(shape=(X_train.shape[1],),name='word_in')
emb_word=(Embedding(len(vocab), EMBED_DIM//4, weights = [embedding_matrix], mask_zero=True))(word_in)

print(X_train.shape[1])

369


In [ ]:
# charcter embedding
char_in = Input(shape=(X_char_train.shape[1],X_char_train.shape[2]), name='char_in')
emb_char=(TimeDistributed(Embedding(input_dim=len(characters),output_dim=10,input_length=max_len_char,mask_zero=True)))(char_in)
char_enc=(TimeDistributed(LSTM(units=EMBED_DIM//4,return_sequences=False)))(emb_char)

# pos_tag embedding 
pos_in=Input(shape=(x_pos_train.shape[1],),name='pos_in')
pos_word=(Embedding(len(pos_tags), EMBED_DIM//2, mask_zero=True))(pos_in)

# concatenating them word+char+pos
x = concatenate([emb_word, char_enc, pos_word])



In [ ]:
#passing to Bi-LSTM layer
o=(Bidirectional(LSTM(BiRNN_UNITS // 2, return_sequences=True, dropout=0.2)))(x)
#print(o)

#Self attention layer
o = (SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       kernel_regularizer=keras.regularizers.l2(1e-4),
                       bias_regularizer=keras.regularizers.l1(1e-4),
                       attention_regularizer_weight=1e-4,
                       name='Attention'))(o)

In [ ]:
#CRF layer
crf = CRF(len(class_labels), sparse_target=True,name='crf')
o=(crf)(o)
model=Model([word_in,char_in,pos_in],o)
model.compile('adam', loss=crf_loss, metrics=[crf_viterbi_accuracy])

In [ ]:
EPOCHS = 5
model.fit([X_train,np.array(X_char_train).reshape(len(X_char_train),max_len,max_len_char),x_pos_train.reshape(len(x_pos_train),max_len)],np.array(y_train).reshape(len(y_train), max_len, 1),batch_size=32, epochs=EPOCHS, validation_split=0.1, verbose=1)
model.save('/content/drive/My Drive/explo/bhoj_explo_chunk.bin')

Train on 3985 samples, validate on 443 samples
Epoch 1/5
3985/3985 [==============================] - 1390s 349ms/step - loss: 172.3186 - crf_viterbi_accuracy: 0.5177 - val_loss: 144.4021 - val_crf_viterbi_accuracy: 0.7409
Epoch 2/5
3985/3985 [==============================] - 1365s 342ms/step - loss: 171.4978 - crf_viterbi_accuracy: 0.8115 - val_loss: 144.0795 - val_crf_viterbi_accuracy: 0.8923
Epoch 3/5
3985/3985 [==============================] - 1338s 336ms/step - loss: 171.2128 - crf_viterbi_accuracy: 0.9069 - val_loss: 143.9287 - val_crf_viterbi_accuracy: 0.9218
Epoch 4/5
3985/3985 [==============================] - 1336s 335ms/step - loss: 171.1123 - crf_viterbi_accuracy: 0.9245 - val_loss: 143.8649 - val_crf_viterbi_accuracy: 0.9241
Epoch 5/5
3985/3985 [==============================] - 1368s 343ms/step - loss: 171.0722 - crf_viterbi_accuracy: 0.9363 - val_loss: 143.8416 - val_crf_viterbi_accuracy: 0.9283


In [ ]:
#validation
model_json = model.to_json()
with open("model_bhoj.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
# model1_new.save_weights("/content/drive/My Drive/explo/model_bhoj_fine.h5")
# print("Saved model to disk")
l4=[0]*max_len_char
l4=numpy.asarray(l4)
y= model.predict([X_test,np.array(X_char_test).reshape((len(X_char_test),max_len, max_len_char)),x_pos_test.reshape(len(x_pos_test),max_len)])
y= y.argmax(-1)
y_pred=[]
test_y_true=[]
ctest_y_pred=[]
ctest_y_true=[]

#removing padding from the validation
for i in range(len(X_test)):
    l=[]
    l1=[]
    for j in range(len(X_test[i])):
        if (X_test[i][j]==0):
            continue
        l.append(y[i][j])
        l1.append(y_test[i][j][0])
        ctest_y_pred.append(y[i][j])
        ctest_y_true.append(y_test[i][j][0])
    y_pred.append(l)
    test_y_true.append(l1)

#converting to numpy array
test_y_pred=numpy.asarray(y_pred)
test_y_true=numpy.asarray(test_y_true)

#writing output for validation
f1=open(output_file,'w')
for i in range(len(test_y_pred)):
    for j in range(len(test_y_pred[i])):
        s='|'+'\t'+'NN'+'\t'+str(class_labels[test_y_pred[i][j]])+'\t'+str(class_labels[test_y_true[i][j]])
        f1.write(s+'\n')
    f1.write('\n')
f1.flush()
f1.close()
print('\n---- Result of BiLSTM-CRF ----\n')
classification_report(ctest_y_true, ctest_y_pred, class_labels)


---- Result of BiLSTM-CRF ----

                   recall precision  f1-score   support

B-BLK                0.67      0.85      0.75       355
B-CCP                0.99      0.99      0.99       747
B-JJP                0.02      0.60      0.04       160
B-NEGP               0.00      0.00      0.00        73
B-NNP                0.00      0.00      0.00         0
B-NP                 0.97      0.93      0.95      6846
B-NST                0.00      0.00      0.00         0
B-PRP                0.00      0.00      0.00         0
B-RBP                0.86      0.78      0.82        81
B-RP                 0.00      0.00      0.00         0
B-VGF                0.98      0.86      0.92      3205
B-VGINF              0.00      0.00      0.00         9
B-VGNF               0.00      0.00      0.00       394
B-VGNN               0.00      0.00      0.00         4
I-BLK                0.44      0.65      0.53        63
I-CCP                0.00      0.00      0.00        11
I-JJP         

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
